In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn import metrics

import os, sys
from time import time

from phm08ds.models import experiment

## Load Dataset

In [2]:
folderpath = '../../../data/interim/'
data_op_05 = pd.read_csv(folderpath + 'data_op_05.csv')
data_op_05.head()

,unit,time_step,operational_setting_1,operational_setting_2,operational_setting_3,Sensor_2,Sensor_3,Sensor_4,Sensor_7,Sensor_11,Sensor_12,Sensor_15,Operational_condition,Health_state
10,1,11,25.0051,0.6200,80.0,1260.50,1051.56,7.05,1915.28,165.53,2028.24,0.02,5,1
13,1,14,25.0073,0.6203,80.0,1262.80,1051.01,7.05,1915.24,164.63,2028.14,0.02,5,1
18,1,19,25.0018,0.6200,80.0,1255.21,1053.69,7.05,1915.22,165.27,2028.13,0.02,5,1
26,1,27,25.0031,0.6206,80.0,1267.93,1042.92,7.05,1915.28,164.90,2028.16,0.02,5,2
28,1,29,25.0026,0.6200,80.0,1256.53,1048.20,7.05,1915.25,164.51,2028.14,0.02,5,2


## Data preprocessing

### Get rid of informations there are not sensor readings

Wang (2008) reports Sensor 15 has importat information. However, there are no relevant informations of this sensor. The data seems to be corrupted like this:

Let's remove it from our database creating an object transformer.

In [3]:
from phm08ds.features.feature_selection import RemoveSensor

tf_remove_sensor_15 = RemoveSensor(sensors=[15])
data_op_05 = tf_remove_sensor_15.fit_transform(data_op_05)
data_op_05.head()

,unit,time_step,operational_setting_1,operational_setting_2,operational_setting_3,Sensor_2,Sensor_3,Sensor_4,Sensor_7,Sensor_11,Sensor_12,Operational_condition,Health_state
10,1,11,25.0051,0.6200,80.0,1260.50,1051.56,7.05,1915.28,165.53,2028.24,5,1
13,1,14,25.0073,0.6203,80.0,1262.80,1051.01,7.05,1915.24,164.63,2028.14,5,1
18,1,19,25.0018,0.6200,80.0,1255.21,1053.69,7.05,1915.22,165.27,2028.13,5,1
26,1,27,25.0031,0.6206,80.0,1267.93,1042.92,7.05,1915.28,164.90,2028.16,5,2
28,1,29,25.0026,0.6200,80.0,1256.53,1048.20,7.05,1915.25,164.51,2028.14,5,2


Before feeding to the classifier, let's remove unwanted information, such as unit, time_step and operational settings.

In [4]:
from phm08ds.features.feature_selection import RemoveInfo

tf_remove_info = RemoveInfo()

data_with_features = tf_remove_info.fit_transform(data_op_05)
data_with_features.head()

,Sensor_2,Sensor_3,Sensor_4,Sensor_7,Sensor_11,Sensor_12,Health_state
10,1260.50,1051.56,7.05,1915.28,165.53,2028.24,1
13,1262.80,1051.01,7.05,1915.24,164.63,2028.14,1
18,1255.21,1053.69,7.05,1915.22,165.27,2028.13,1
26,1267.93,1042.92,7.05,1915.28,164.90,2028.16,2
28,1256.53,1048.20,7.05,1915.25,164.51,2028.14,2


We need to normalize our data. Let's use Z-score standardization.

In [5]:
from sklearn.preprocessing import StandardScaler

tf_std_scaller = preprocessing.StandardScaler()
data_with_features_std = tf_std_scaller.fit_transform(data_with_features.drop(labels='Health_state', axis=1))
data_with_features_std

array([[-4.39001956e-01,  1.46599696e-01,  1.00000000e+00,
        -4.59248267e-01,  2.83733641e+00, -2.09045391e-01],
       [-6.08852380e-04,  6.72683790e-02,  1.00000000e+00,
        -6.11506778e-01,  2.00798211e-01, -5.69484735e-01],
       [-1.44730609e+00,  4.53828253e-01,  1.00000000e+00,
        -6.87636034e-01,  2.07566982e+00, -6.05528670e-01],
       ...,
       [ 1.47086713e+00,  2.48326759e+00,  1.00000000e+00,
        -1.86763949e+00, -2.38624822e-01, -1.68684670e+00],
       [ 5.94080923e-01,  2.74001258e+00,  1.00000000e+00,
        -1.75344561e+00, -1.00029141e+00, -1.83102244e+00],
       [ 2.25806866e+00,  2.62894874e+00,  1.00000000e+00,
        -2.05796263e+00, -1.11747089e+00, -1.97519818e+00]])

In [6]:
labels = np.array(data_with_features['Health_state'])
labels

array([1, 1, 1, ..., 4, 4, 4])

# Classification steps

In [7]:
from phm08ds.models import experiment

## Load best mlp from random search

In [8]:
from sklearn.externals import joblib

random_search_mlp = joblib.load('Results/old_models/clf_mlp.pkl')
# random_search_svm = joblib.load('Results/old_models/clf_svm.pkl')

In [9]:
clf_mlp = random_search_mlp['MLP'].best_estimator_

## Put all clf in a dictionary:

In [10]:
classifiers = {'MLP': clf_mlp}

Since we are using SVM and MLP we need to extract all power from those methods. Let's perform a Random Search to parameters optimizations.

In [11]:
kfolds = 10
clf_outputs = experiment.run_classifiers(data_with_features_std, labels, classifiers, kfolds)

## Performance assessment

Saving variables in a dictionary:

In [12]:
results = {}
results['train'] = experiment.results_clf(4, clf_outputs['train']['true'], clf_outputs['train']['pred'])
results['test'] = experiment.results_clf(4, clf_outputs['test']['true'], clf_outputs['test']['pred'])

## Savel results, models and pipeline to a .pkl file 

In [13]:
from sklearn.pipeline import Pipeline

data_preprocessing = Pipeline([('remove_sensor_15', tf_remove_sensor_15),
                               ('remove_info', tf_remove_info),
                               ('std_scaler', tf_std_scaller)
                              ])

In [14]:
experiment.save_models(classifiers)
experiment.save_pipeline(data_preprocessing)

## Save results to CSVs and figures

In [15]:
experiment.export_results(results['test'], 'test')
experiment.export_results(results['train'], 'train')

/mnt/Work/Mestrado/Mestrado/MP-Safety_ITA/Analyses_and_Experiments/PHM08_data_science/notebooks/E03_PHM08-train-best_MLP_and_SVM/model_selection-OP_05-navarmn
MLP
/mnt/Work/Mestrado/Mestrado/MP-Safety_ITA/Analyses_and_Experiments/PHM08_data_science/notebooks/E03_PHM08-train-best_MLP_and_SVM/model_selection-OP_05-navarmn
MLP


## Savel results, models and pipeline to a .pkl file 

In [16]:
from sklearn.pipeline import Pipeline

data_preprocessing = Pipeline([('remove_sensor_15', tf_remove_sensor_15),
                               ('remove_info', tf_remove_info),
                               ('std_scaler', tf_std_scaller)
                              ])

In [17]:
experiment.save_models(clf_mlp, name='clf_mlp')
experiment.save_pipeline(data_preprocessing)